In [1]:
import polars as pl
import numpy as np
import time
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import KFold
from imblearn.under_sampling import RandomUnderSampler

In [2]:
df_polars_raiz = pl.read_parquet('../dataset.parquet')

In [3]:
df_polars = df_polars_raiz.sample(fraction=0.01, seed=42)

In [4]:
import ipaddress

def ip_to_int(ip: str) -> int:
    try:
        return int(ipaddress.ip_address(ip))  # Funciona tanto para IPv4 quanto IPv6
    except ValueError:
        return None

In [5]:
#df_polars = df_polars.with_columns([
#    pl.col('id.resp_h').map_elements(ip_to_int).alias('id.resp_h'),
#    pl.col('id.orig_h').map_elements(ip_to_int).alias('id.orig_h')
#])

In [6]:
df_polars = df_polars.with_columns([
    pl.col('duration').fill_null(0),
    pl.col('orig_bytes').fill_null(0),
    pl.col('resp_bytes').fill_null(0)
])

In [7]:
lista_colunas = df_polars.columns
colunas_para_spearman = ['id.resp_p', 'history', 'conn_state', 'id.orig_p', 'orig_ip_bytes', 'label']             
#['detailed-label', 'id.resp_p', 'history', 'id.orig_h', 'conn_state', 'id.orig_p', 'orig_ip_bytes']
colunas_para_dropar = [col for col in lista_colunas if col not in colunas_para_spearman]
df_polars = df_polars.drop(colunas_para_dropar)

In [8]:
df_polars = df_polars.drop_nulls()

In [9]:
X = df_polars.drop('label')
y = df_polars['label']       

In [10]:
#X = X.to_numpy()
#X[:, 0] = np.array([ip_to_int(ip) for ip in X[:, 0]])

# Treinamento

In [11]:
results = []

In [12]:
def startTrain(X, y, penalty, C, solver):
    kfold = KFold(n_splits=5, shuffle=True)
    results_fold = []

    for train_idx, test_idx in kfold.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # 🔹 Normalização MinMaxScaler
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

        # Aplicando o undersampling no conjunto de treino
        #rus = RandomUnderSampler()
        #X_train_resampled, y_train_resampled = rus.fit_resample(X_train_scaled, y_train)

        start_training = time.time()

        # Inicializa o modelo de Regressão Logística
        lr = LogisticRegression(penalty=penalty, C=C, solver=solver)
        
        # Treina o modelo
        lr.fit(X_train_scaled, y_train)
        end_training = time.time()

        # Predição
        y_pred = lr.predict(X_test_scaled)
        evaluation_duration = time.time() - end_training
        training_duration = end_training - start_training
        
        confusion = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = confusion.ravel()
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        specificity = tn / (tn + fp)
        f1 = f1_score(y_test, y_pred)
        balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
        false_alarm_rate = fp / (fp + tn) if (fp + tn) > 0 else 0
        
        avaliacao = [accuracy, balanced_accuracy, precision, recall, specificity, f1, false_alarm_rate, tn, fp, fn, tp, training_duration, evaluation_duration]
        #print(avaliacao)
        results_fold.append(avaliacao)

    results_fold_array = np.array(results_fold, dtype=np.float32)
    mean_results = np.mean(results_fold_array, axis=0)
    results.append(["LR"] + mean_results.tolist())

In [13]:
#Melhores Hiperparâmetros encontrados: {'penalty': 'l2', 'C': 1, 'solver': 'newton-cg'}

In [14]:
for i in range(1,11):
    startTrain(X=X, y=y, penalty='l2', C=1, solver='newton-cg')
    print(i)

1
2
3
4
5
6
7
8
9
10


In [15]:
metrics_df = pl.DataFrame(
    results,
    schema=['Algorithm', 'Accuracy', 'Balanced Accuracy', 'Precision', 'Recall', 'Specificity', 'F1-score', 'False Alarm Rate', 'tn', 'fp', 'fn', 'tp', 'training_duration', 'evaluation_duration']
)
metrics_df

c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\functools.py:888: DataOrientationWarning: Row orientation inferred during DataFrame construction. Explicitly specify the orientation by passing `orient="row"` to silence this warning.
  return dispatch(args[0].__class__)(*args, **kw)


Algorithm,Accuracy,Balanced Accuracy,Precision,Recall,Specificity,F1-score,False Alarm Rate,tn,fp,fn,tp,training_duration,evaluation_duration
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""LR""",0.979893,0.96905,0.990989,0.985011,0.953088,0.987991,0.046912,16968.0,835.200012,1397.599976,91845.796875,1.051911,0.003257
"""LR""",0.979902,0.969078,0.990999,0.985012,0.953145,0.987996,0.046855,16969.0,834.200012,1397.599976,91845.796875,1.141673,0.003518
"""LR""",0.979899,0.969071,0.990997,0.985009,0.953133,0.987994,0.046866,16968.800781,834.400024,1397.800049,91845.601562,1.06284,0.003035
"""LR""",0.9799,0.969074,0.990999,0.985009,0.95314,0.987995,0.04686,16969.0,834.200012,1397.800049,91845.601562,1.040117,0.002806
"""LR""",0.979895,0.969062,0.990995,0.985007,0.953118,0.987992,0.046882,16968.599609,834.599976,1398.0,91845.398438,0.908792,0.003582
"""LR""",0.979899,0.969063,0.990995,0.985011,0.953115,0.987994,0.046885,16968.599609,834.599976,1397.599976,91845.796875,1.05615,0.004201
"""LR""",0.97989,0.969041,0.990986,0.985009,0.953072,0.987989,0.046928,16967.800781,835.400024,1397.800049,91845.601562,0.992062,0.003294
"""LR""",0.979897,0.969057,0.990993,0.985011,0.953104,0.987993,0.046896,16968.400391,834.799988,1397.599976,91845.796875,0.875682,0.002044
"""LR""",0.9799,0.969073,0.990997,0.985011,0.953134,0.987995,0.046866,16968.800781,834.400024,1397.599976,91845.796875,0.961263,0.003553


In [16]:
metrics_df.write_csv("metrics_results/unbalanced_lr_metrics_output.csv", separator=';')